In [11]:
#SVM
import numpy as np 
import pandas as pd 
from scipy.io import loadmat
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random as rnd


mat_contents = loadmat('data5.mat')
data = mat_contents['x']
np.random.shuffle(data)
# normalization
def normalize(x):
  xmean = np.mean(x, axis=0)
  xstd = np.std(x)
  ret = (x - xmean)/(xstd)  
  return ret

N, M = data.shape
data = data[:, 1:]
np.random.shuffle(data)
N, M = data.shape
ones = np.ones((N, 1))
x = data[:, :M-1]
x = normalize(x)
X = x
X = np.concatenate((ones, X), axis=1)
Y = data[:, M-1:]

for i in range(N):
  if (Y[i][0] == 0):
    Y[i][0] = -1

train_size = int(0.7*N)
validation_size = int(0.1*N)
trainX = X[:train_size, :]
trainY = Y[:train_size, :]
valX = X[train_size:train_size+validation_size, :]
valY = Y[train_size:train_size+validation_size, :]
testX = X[train_size+validation_size:, :]
testY = Y[train_size+validation_size:, :]

def kernel(x1, x2, ker):
  if ker == 'linear':
    return np.dot(x1, x2.T)

  elif ker == 'quad':
    return np.dot(x1, x2.T)**2

  elif ker == 'gauss':
    return np.exp(- (np.linalg.norm(x1 - x2, 2)) ** 2 / (2))

def calc_w(x, y, alpha):
  return np.dot(x.T, np.multiply(alpha, y))

def calc_b(x, y, w):
  return np.mean(y - np.dot(w.T, x.T))

def compute_L_H(C, al_j, al_i, y_j, y_i):
  if (y_i != y_j):
      return (max(0, al_j - al_i), min(C, C - al_i + al_j))
  else:
      return (max(0, al_i + al_j - C), min(C, al_i + al_j))

def rand_int(a, b, z):
  i = z
  count = 0
  while i == z and count < 1000:
      i = rnd.randint(a,b)
      count = count+1
  return i

def h(X, w, b):
  return np.sign(np.dot(w.T, X.T) + b).astype(int)

def E(x_k, y_k, w, b):
  return h(x_k, w, b) - y_k

def predict(w, X, b):
  return h(X, w, b)



def SVM(x, y, ker, epsilon, max_iter, C):
  n, m = x.shape
  alpha = np.zeros((n, 1))
  count = 0

  for i in range(max_iter-1):
    count += 1
    alpha_prev = np.copy(alpha)
    for j in range(n):
      i = rand_int(0, n-1, j)
      xi, xj, yi, yj = x[i, :], x[j, :], y[i][0], y[j][0]
      kij = kernel(xi, xi, ker) + kernel(xj, xj, ker) - 2*kernel(xi, xj, ker)
      if kij == 0:
        continue

      al_i, al_j = alpha[i], alpha[j]
      (L, H) = compute_L_H(C, al_j, al_i, yj, yi)
      w = calc_w(x, y, alpha)
      b = calc_b(x, y, w)

      E_i = E(xi, yi, w, b)
      E_j = E(xj, yj, w, b)
      alpha[j] = al_j + float(yj * (E_i - E_j)) / kij
      alpha[j] = max(alpha[j], L)
      alpha[j] = min(alpha[j], H)
      alpha[i] = al_i + yi*yj * (al_j - alpha[j])
    diff = np.linalg.norm(alpha - alpha_prev)
    if diff < epsilon:
        break

  b = calc_b(x, y, w)
  if ker == 'linear':
      w = calc_w(x, y, alpha)
  alpha_idx = np.where(alpha > 0)[0]
  support_vectors = x[alpha_idx, :]
  return support_vectors, count, w, b

support_vec, count, w, b = SVM(trainX, trainY, 'gauss', 0.1, 10, 1)
pred_y = predict(w, testX, b)
n, m = testY.shape
tp, tn, fp, fn = 0,0,0,0
for i in range(n):
  if (trainY[i][0] == 1.0) & (pred_y[0][i] == 1.0):
      tp += 1
  elif (trainY[i][0] == 1.0) & (pred_y[0][i] == 0.0):
      fn += 1
  elif (trainY[i][0] == 0.0) & (pred_y[0][i] == 1.0):
      fp +=1
  elif (trainY[i][0] == 0.0) & (pred_y[0][i] == 0.0):
      tn += 1

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print('Accuracy: ',accuracy,'sensitivity: ', sensitivity,'specificity: ', specificity)
  

ZeroDivisionError: ignored

In [22]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import scipy.io
from sklearn.preprocessing import normalize
import random as rnd
from __future__ import division, print_function
from sklearn.metrics import confusion_matrix as cm

mat = scipy.io.loadmat('data5.mat')
data = mat['x']
data = np.asarray(data)
np.random.shuffle(data)
len = data.shape[0]

train = int(0.7 * len)
val = int((0.1 * len) + 1)
test = int((0.2 * len) + 1)
train_data = data[0 : train,:]
val_data = data[train+1 : train+1+val,:]
test_data = data[train+val+1 :,:]

x_train = train_data[:,0:72]
y_train = train_data[:,-1]
x_val = val_data[:,0:72]
y_val = val_data[:,-1]
x_test = test_data[:,0:72]
y_test = test_data[:,-1]

for i in range(y_train.shape[0]):
  y_train[i] = int(y_train[i])
  if int(y_train[i]) == 0:
    y_train[i] = -1
for i in range(y_val.shape[0]):
  y_val[i] = int(y_val[i])
  if int(y_val[i]) == 0:
    y_val[i] = -1
for i in range(y_test.shape[0]):
  y_test[i] = int(y_test[i])
  if int(y_test[i]) == 0:
    y_test[i] = -1

class SVM():
    def __init__(self, max_iter=5000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon

    def fit(self, X, y):
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                i = self.get_rnd_int(0, n-1, j) 
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if count >= self.max_iter:
                return

        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)

        alpha_ind = np.where(alpha > 0)[0]
        support_vectors = X[alpha_ind, :]
        return support_vectors, count

    def predict(self, X):
        return self.h(X, self.w, self.b)

    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)

    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))

    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)

    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k

    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, yj, yi):
        if(yi != yj):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))

    def get_rnd_int(self, a,b,z):
        i = z
        count=0
        while i == z and count<1000:
            i = rnd.randint(a,b)
            count += 1
        return i

    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)

    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)

def print_acc(y_pred,y_actual):
    confmat = cm(y_actual, y_pred)
    Accuracy = (confmat[0][0]+confmat[1][1])/ (np.shape(y_pred)[0])
    Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    
    print(confmat)
    print("\n")
    print(f"Accuracy: {Accuracy*100}%\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}\n")

svm = SVM()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)

print_acc(y_pred,y_test)

[[163  53]
 [ 23 190]]


Accuracy: 82.28438228438229%
Sensitivity: 0.892018779342723
Specificity: 0.7546296296296297

